In [2]:
# Generated synthetic univariate or multivariate time series data
import pandas as pd
import numpy as np
from einops import rearrange

def generate_time_series_data(start_time: str = "2021-01-01 00:00:00", 
                              freq: str = "H", 
                              periods: int = 40, 
                              n_variables: int = 10, 
                              univariate: bool = False) -> pd.DataFrame:
    """
    Generate a univariate or multivariate time series DataFrame.
    
    Args:
        start_time (str): Start datetime.
        freq (str): Pandas frequency string (e.g., 'H' for hourly).
        periods (int): Number of time steps.
        n_variables (int): Number of variables (ignored if univariate=True).
        univariate (bool): Whether to generate univariate (single-column) data.

    Returns:
        pd.DataFrame: Time series data with datetime index.
    """
    index = pd.date_range(start=start_time, periods=periods, freq=freq)

    if univariate:
        data = np.random.randn(periods)
        df = pd.DataFrame(data, index=index, columns=["A"])
    else:
        columns = list("ABCDEFGHIJ")[:n_variables]
        data = np.random.randn(periods, n_variables)
        df = pd.DataFrame(data, index=index, columns=columns)
    
    return df

df_multivariate = generate_time_series_data()
print(df_multivariate.columns.values)
df_univariate = generate_time_series_data(univariate=True)
print(df_univariate.head())

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J']
                            A
2021-01-01 00:00:00 -1.784323
2021-01-01 01:00:00 -0.516175
2021-01-01 02:00:00  0.437445
2021-01-01 03:00:00 -0.590988
2021-01-01 04:00:00  0.710985


In [17]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.model.moirai_moe import MoiraiMoEForecast, MoiraiMoEModule

# Model configuration
MODEL = "moirai"  # model name: choose from {'moirai', 'moirai-moe'}
SIZE = "small"  # model size: choose from {'small', 'base', 'large'}
PDT = 60  # prediction length: any positive integer
CTX = 40  # context length: any positive integer
PSZ = 32  # patch size: choose from {"auto", 8, 16, 32, 64, 128}
BSZ = 8  # batch size: any positive integer
TEST = 8  # test set length: any positive integer



In [18]:
def moirai_maker(MODEL):
    if MODEL == "moirai":
        model = MoiraiForecast(
            module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.1-R-{SIZE}"),
            prediction_length=PDT,
            context_length=CTX,
            patch_size=PSZ,
            num_samples=500,
            target_dim=1,
            feat_dynamic_real_dim=0,
            past_feat_dynamic_real_dim=0,
        )
    elif MODEL == "moirai-moe":
        model = MoiraiMoEForecast(
            module=MoiraiMoEModule.from_pretrained(f"Salesforce/moirai-moe-1.0-R-{SIZE}"),
            prediction_length=PDT,
            context_length=CTX,
            patch_size=PSZ,
            num_samples=500,
            target_dim=1,
            feat_dynamic_real_dim=0,
            past_feat_dynamic_real_dim=0,
        )
    return model

In [19]:
# Moirai univariate
model = moirai_maker("moirai")

# Time series values. Shape: (batch, time, variate)
past_target = rearrange(
    torch.as_tensor(df_univariate["A"].to_numpy(), dtype=torch.float32), "t -> 1 t 1"
)
# 1s if the value is observed, 0s otherwise. Shape: (batch, time, variate)
past_observed_target = torch.ones_like(past_target, dtype=torch.bool)
# 1s if the value is padding, 0s otherwise. Shape: (batch, time)
past_is_pad = torch.zeros_like(past_target, dtype=torch.bool).squeeze(-1)

#print("past_target.shape:", past_target.shape)
#print("past_observed_target.shape:", past_observed_target.shape)
#print("past_is_pad.shape:", past_is_pad.shape)
print("moirai start")
forecast = model(
    past_target=past_target,
    past_observed_target=past_observed_target,
    past_is_pad=past_is_pad,
)
print("moirai end")


print(np.round(np.median(forecast[0], axis=0), decimals=4),)




moirai start
moirai end
[0.9221 0.8047 0.8664 1.0779 1.1799 1.2292 1.0917 0.9621 0.779  0.819
 0.8958 1.1054 1.166  1.105  1.1139 1.0751 1.2097 1.2432 1.2658 1.079
 1.025  0.9428 0.9036 1.0599 1.0931 1.1266 1.2288 1.0618 1.1765 1.1986
 1.1631 1.1768 0.642  0.7636 0.629  0.5958 0.5269 0.5235 0.6213 0.7105
 0.615  0.6891 0.5689 0.7422 0.6613 0.5758 0.5737 0.5537 0.5627 0.584
 0.6588 0.6301 0.8834 0.8627 0.7345 0.6447 0.8316 0.62   0.5919 0.6872]


In [20]:
# Moirai-MoE univariate
model = moirai_maker("moirai-moe")

# Time series values. Shape: (batch, time, variate)
past_target = rearrange(
    torch.as_tensor(df_univariate["A"].to_numpy(), dtype=torch.float32), "t -> 1 t 1"
)
# 1s if the value is observed, 0s otherwise. Shape: (batch, time, variate)
past_observed_target = torch.ones_like(past_target, dtype=torch.bool)
# 1s if the value is padding, 0s otherwise. Shape: (batch, time)
past_is_pad = torch.zeros_like(past_target, dtype=torch.bool).squeeze(-1)

#print("past_target.shape:", past_target.shape)
#print("past_observed_target.shape:", past_observed_target.shape)
#print("past_is_pad.shape:", past_is_pad.shape)
print("moirai_moe start")
forecast = model(
    past_target=past_target,
    past_observed_target=past_observed_target,
    past_is_pad=past_is_pad,
)
print("moirai_moe end")


print(np.round(np.median(forecast[0], axis=0), decimals=4),)


moirai_moe start
moirai_moe end
[0.0151 0.0151 0.0364 0.0156 0.0156 0.0156 0.0157 0.0162 0.0152 0.0395
 0.0158 0.0159 0.0157 0.0162 0.0155 0.0152 0.5208 0.4502 0.4323 0.4616
 0.4438 0.4622 0.44   0.4999 0.455  0.4693 0.512  0.4509 0.4609 0.4282
 0.4532 0.438 ]
